# Blackjack - Q-Learning

Este codigo esta basado en el ambiente que proporciona OpenAI en su libreria Gym en codigo [blackjack.py](https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py).

Y el algoritmo Q-Learning esta basado en el que esta descrito en el libro de Reinforcement learning: An introduction (second edition) de Richard S. Sutton y Andrew G. Barto

### Librerias

* Plots se usará para graficar la recompensa a travez de los pasos
* Distributions es para tener la distribución binomial

In [2]:
import Plots
import Distributions
Plots.pyplot();
import Base: step, reset

### Reglas
A continuación se definiran las reglas para el juego y funciones para ejercerlas

In [3]:
# 1 = Ace, 2-10 = Number cards, Jack/Queen/King = 10
deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]

function cmp_(a, b)
    return Float64(a > b) - Float64(a < b)
end


function draw_card()
    return Int(rand(deck))
end


function draw_hand()
    return [draw_card(), draw_card()]
end


function usable_ace(hand)  # Does this hand have a usable ace?
    return 1 in hand && sum(hand) + 10 <= 21
end


function sum_hand(hand)  # Return current hand total
    if usable_ace(hand)
        return sum(hand) + 10
    end
    return sum(hand)
end


function is_bust(hand)  # Is this hand a bust?
    return sum_hand(hand) > 21
end
    
function score(hand)  # What is the score of this hand (0 if bust) 
    if is_bust(hand) return 22 else sum_hand(hand) end
end


function is_natural(hand)  # Is this hand a natural blackjack?
    return sort(hand) == [1, 10]
end


is_natural (generic function with 1 method)

### Estructura del ambiente del BlackJack

Esta estructura es la que contiene el estado del juego y la función de paso (step)

In [4]:
mutable struct BlackjackEnv
    actions::Array{Int64,1}
    natural::Bool
    dealer::Array{Int64, 1}
    player::Array{Int64, 1}
    BlackjackEnv(n = false, player=draw_hand(), dealer=draw_hand()) = new(
        [1,2],
        n,
        player,
        dealer)
end

function _get_obs(self::BlackjackEnv)
    return (sum_hand(self.player), sum_hand(self.dealer), usable_ace(self.player))
end

function reset(self::BlackjackEnv)
    self.dealer = draw_hand()
    self.player = draw_hand()
    return _get_obs(self)
end

function step(self::BlackjackEnv, action::Int)
    @assert action ∈ self.actions "Action must be 2 (hit) or 1 (stick)"
    
    # Because we don't want to modify the original self (we need the original self for the algorithm)
    new_self = deepcopy(self)
    if Bool(action-1)  # hit: add a card to players hand and return
        append!(new_self.player, draw_card())
        if is_bust(new_self.player)
            done = true
            reward = -1
        else
            done = false
            reward = 0
        end
    else   # stick: play out the dealers hand, and score
        done = true
        while sum_hand(new_self.dealer) < 17
            append!(new_self.dealer, draw_card())
        end
        reward = cmp_(score(new_self.player), score(new_self.dealer))
        if new_self.natural && is_natural(new_self.player) && reward == 1
            reward = 1.5
        end
    end
    return (new_self, _get_obs(new_self), reward, done, Dict())
end

function is_start_terminal(self::BlackjackEnv)
    p = score(self.player)
    if p == 0 || p == 21
        done = true
    else
        done = false
    end
    return done
end

is_start_terminal (generic function with 1 method)

### Escoger acción

In [5]:
# probability for exploration
ε = 0.1

# step size
α = 0.5

# gamma for Q-Learning and Expected Sarsa
γ = 1

# choose an action based on epsilon greedy algorithm
function choose_action(state, q_value)
    if rand(Distributions.Binomial(1, ε)) == 1
        #println("Entro al random")
        action = rand(state.actions)
    else
        values_ = q_value[score(state.player), state.dealer[1], :]
        action = rand([action_ for (action_, value_) in enumerate(values_) if value_ == maximum(values_)])
    end
    return action
end

choose_action (generic function with 1 method)

### Q-Learning

In [29]:
# an episode with Q-Learning
# @q_value: values for state action pair, will be updated
# @step_size: step size for updating
# @return: total rewards within this episode
function q_learning(q_value, step_size=α)
    state = BlackjackEnv(true)
    rewards = 0.0
    done = is_start_terminal(state)
    if done == true
        if score(state.player) == 21
            rewards == 500
        end
    end
    while !done  #state != GOAL
        action = choose_action(state, q_value)
        next_state, obs, reward, done, _ = step(state, action)
        rewards += reward
        
        #println("state: ", typeof(state))
        #println("next_state: ", typeof(next_state))
        #println("action: ", typeof(action))
        # Q-Learning update
        q_value[score(state.player), state.dealer[1], action] += step_size * (
                reward + γ * maximum(q_value[score(next_state.player),next_state.dealer[1], :]) -
                q_value[score(state.player), state.dealer[1], action])
        state = next_state
    end
    return rewards
end

q_learning (generic function with 2 methods)

In [21]:
function print_optimal_policy(q_value)
    optimal_policy = []
    for i in range(2, stop=22)
        push!(optimal_policy,[])
        for j in range(1, stop=10)
            if i >= 21
                push!(optimal_policy[end], (i,j,"End"))
                continue
            end
            bestAction = argmax(q_value[i, j, :])
            if bestAction == 2
                push!(optimal_policy[end], (i,j,"Hit"))
            elseif bestAction == 1
                push!(optimal_policy[end], (i,j,"Stick"))
            end
        end
    end
    #println(optimal_policy)
    for row in optimal_policy
        println(row)
    end
    #return optimal_policy
end

print_optimal_policy (generic function with 1 method)

### Prueba

In [30]:
# Use multiple runs instead of a single run and a sliding window
# With a single run I failed to present a smooth curve
# However the optimal policy converges well with a single run
# Sarsa converges to the safe path, while Q-Learning converges to the optimal path

# episodes of each run
episodes = 1000000

# perform 40 independent runs
runs = 40

rewards_q_learning = zeros(episodes)

# 1 index - Score from the player
# 2 index - First card from the dealer (because we don't know the other card)
# 3 index - Number of actions
q_q_learning = zeros(Float16, 22, 10, 2)

for r in range(1, stop=runs)
    q_q_learning = zeros(Float16, 22, 10, 2)
    for i in range(1, stop=episodes)
        # cut off the value by -100 to draw the figure more elegantly
        # rewards_sarsa[i] += max(sarsa(q_sarsa), -100)
        # rewards_q_learning[i] += max(q_learning(q_q_learning), -100)
        rewards_q_learning[i] += q_learning(q_q_learning)
    end
end

# averaging over independt runs
rewards_q_learning /= runs
sort(rewards_q_learning)

1000000-element Array{Float64,1}:
 -1.0  
 -1.0  
 -1.0  
 -1.0  
 -1.0  
 -1.0  
 -1.0  
 -1.0  
 -1.0  
 -1.0  
 -1.0  
 -1.0  
 -1.0  
  ⋮    
 -0.2  
 -0.2  
 -0.2  
 -0.2  
 -0.2  
 -0.2  
 -0.2  
 -0.2  
 -0.2  
 -0.175
 -0.175
 -0.15 

In [31]:
println("Q-Learning Optimal Policy:")
# (Puntaje del jugador, Carta visible del dealer, Accion del jugador)
print_optimal_policy(q_q_learning)

Q-Learning Optimal Policy:
Any[(2, 1, "Stick"), (2, 2, "Stick"), (2, 3, "Stick"), (2, 4, "Stick"), (2, 5, "Stick"), (2, 6, "Stick"), (2, 7, "Stick"), (2, 8, "Stick"), (2, 9, "Stick"), (2, 10, "Stick")]
Any[(3, 1, "Stick"), (3, 2, "Stick"), (3, 3, "Stick"), (3, 4, "Stick"), (3, 5, "Stick"), (3, 6, "Stick"), (3, 7, "Stick"), (3, 8, "Stick"), (3, 9, "Stick"), (3, 10, "Stick")]
Any[(4, 1, "Stick"), (4, 2, "Hit"), (4, 3, "Stick"), (4, 4, "Hit"), (4, 5, "Hit"), (4, 6, "Stick"), (4, 7, "Stick"), (4, 8, "Stick"), (4, 9, "Stick"), (4, 10, "Hit")]
Any[(5, 1, "Stick"), (5, 2, "Hit"), (5, 3, "Stick"), (5, 4, "Stick"), (5, 5, "Stick"), (5, 6, "Hit"), (5, 7, "Stick"), (5, 8, "Stick"), (5, 9, "Hit"), (5, 10, "Hit")]
Any[(6, 1, "Stick"), (6, 2, "Stick"), (6, 3, "Hit"), (6, 4, "Hit"), (6, 5, "Hit"), (6, 6, "Stick"), (6, 7, "Stick"), (6, 8, "Stick"), (6, 9, "Stick"), (6, 10, "Stick")]
Any[(7, 1, "Stick"), (7, 2, "Stick"), (7, 3, "Stick"), (7, 4, "Stick"), (7, 5, "Stick"), (7, 6, "Hit"), (7, 7, "Stick"),